In [111]:
import Pkg

Pkg.add("JLD2")

    Updating registry at `C:\Users\matsz\.julia\registries\General.toml`
   Resolving package versions...
   Installed FileIO ─ v1.16.3
   Installed JLD2 ─── v0.4.51
    Updating `C:\Users\matsz\.julia\environments\v1.10\Project.toml`
  [033835bb] + JLD2 v0.4.51
    Updating `C:\Users\matsz\.julia\environments\v1.10\Manifest.toml`
  [5789e2e9] + FileIO v1.16.3
  [033835bb] + JLD2 v0.4.51
Precompiling project...
  ✓ FileIO
  ✓ Plots → FileIOExt
  ✓ JLD2
  ✓ TaylorSeries → TaylorSeriesJLD2Ext
  4 dependencies successfully precompiled in 79 seconds. 538 already precompiled.


In [3]:
using Revise
using Turing, MCMCChains
using XLSX
using DataFrames, TimeSeries
using StatsPlots
using LinearAlgebra
using CSV
using StatsBase
include("utils.jl")
include("financial_bvar_temp.jl")
using .FinancialBVAR
using JLD2

In [4]:
data = TimeArray(CSV.File("usa_data_transformed.csv"),timestamp=:timestamp)
start = Date(1993,03,01)
end_d = Date(2023, 12, 31)
s_data = to(from(data, start),end_d)

124×6 TimeArray{Float64, 2, Date, Matrix{Float64}} 1993-03-01 to 2023-12-01
┌────────────┬─────────────────┬───────────────┬─────────────┬───────────────┬──
│            │ Total_Stock_CPI │ Price_10Y_CPI │ T_Bill_CPI  │ Dividend_Yeld │ ⋯
├────────────┼─────────────────┼───────────────┼─────────────┼───────────────┼──
│ 1993-03-01 │       0.0150658 │     0.0543577 │ -0.00250148 │     0.0281965 │ ⋯
│ 1993-06-01 │       0.0136547 │     0.0292271 │  0.00478562 │     0.0279908 │ ⋯
│ 1993-09-01 │       0.0342312 │       0.03506 │ -0.00169417 │     0.0270317 │ ⋯
│ 1993-12-01 │       0.0226641 │    -0.0219636 │  0.00410946 │     0.0266883 │ ⋯
│ 1994-03-01 │      -0.0571526 │    -0.0622961 │ 0.000425924 │     0.0285162 │ ⋯
│ 1994-06-01 │      0.00958316 │    -0.0293652 │  0.00338014 │     0.0285113 │ ⋯
│ 1994-09-01 │        0.026626 │   -0.00884948 │  0.00390608 │     0.0280574 │ ⋯
│ 1994-12-01 │      0.00488791 │  -0.000941161 │  0.00828514 │     0.0283289 │ ⋯
│     ⋮      │        ⋮        │ 

In [23]:
vmodel = VARModel(s_data)
scenarios = simulate(vmodel,60, 100_000)
returns_scenarios = scenarios[1:3,:,:]

3×100000×61 Array{Float64, 3}:
[:, :, 1] =
 0.121595   0.121595   0.121595   …  0.121595   0.121595   0.121595
 0.0651554  0.0651554  0.0651554     0.0651554  0.0651554  0.0651554
 0.0103466  0.0103466  0.0103466     0.0103466  0.0103466  0.0103466

[:, :, 2] =
  0.119236   0.00769099  -0.00834449  …  0.0292722   0.10863     -0.0454326
 -0.0176562  0.0463184   -0.00728951     0.0755906  -0.00156152  -0.0249447
  0.011914   0.0068984    0.00234649     0.0131597  -0.00259614  -0.00375791

[:, :, 3] =
 0.0337548    0.0954506   0.227331    …  0.114711   0.115749    -0.0272147
 0.0651487   -0.0418501  -0.0147174      0.0266172  0.00710173   0.0938159
 0.00711104   0.0028917   0.00147573     0.0137467  0.00894963   0.0227659

;;; … 

[:, :, 59] =
  0.0148462    0.0499572    0.0691218   …  -0.0502689    -0.0844181
 -0.0770212   -0.114722    -0.0712956       0.00310456    0.0561507
 -0.00735492  -0.00573927   0.00246246     -0.000314397  -0.0120119

[:, :, 60] =
 -0.0545449   -0.101766    0.00

In [24]:
includet("kmeans_moments_lattice.jl")
using .LatticeGeneration

In [8]:
s = sum(returns_scenarios[:,:,21:25]; dims=3)
s = dropdims(s; dims= 3)
mean(s[1,:,:])

5.077523642321002

In [25]:
returns_scenarios_ann = zeros(3,100_000,15)
for asset in 1:3
    returns_scenarios_ann[asset,:,:] = exp.(annualise(returns_scenarios[asset,:,2:end],4))
end
returns_scenarios_ann


3×100000×15 Array{Float64, 3}:
[:, :, 1] =
 1.31453  1.35426   1.58131   0.989565  …  1.13099  1.09402  1.35336  1.173
 1.03274  0.953603  0.853565  0.97904      1.00434  1.13451  1.03421  0.94448
 1.01454  1.00972   0.990088  1.01329      1.02803  1.02315  1.00541  1.01986

[:, :, 2] =
 1.23586   1.05858   0.857927  …  1.0135   1.24546   1.1101   1.0202
 0.922771  0.988084  0.925957     1.17741  0.976392  1.03849  1.01424
 0.996715  0.993874  1.0118       1.0161   0.991691  1.02746  1.03013

[:, :, 3] =
 0.937157  0.748103  1.13864  1.03392   …  1.22937   1.04516  1.18099
 0.959484  1.07704   1.14956  0.931722     0.89361   1.04203  0.951224
 0.970973  1.04851   1.03743  0.99766      0.975253  1.01378  0.995028

;;; … 

[:, :, 13] =
 1.03171   0.897457  1.08127   1.01917   …  0.942035  1.25605   0.984456
 1.09084   0.984182  0.983282  0.991762     1.06735   0.919033  0.950755
 0.990611  0.984049  1.00908   1.01073      1.00431   0.978495  1.00291

[:, :, 14] =
 1.1245    1.05688  0.90

In [26]:
res = generate_lattice(returns_scenarios_ann, 9,15)


Lattice([1.0084913942473595 1.2943722619880673 … 1.2045282066834457 1.0752274448581267; 1.0963921326249169 0.9668353671859581 … 1.0855946827898875 1.3705173432588122; … ; 1.3667152684087012 0.9660598526959102 … 0.9503650809004188 1.120501476035486; 0.9840055595486175 1.3637141036185536 … 1.1632620944457561 1.1207337251940046;;; 0.9336306910725175 1.026510299706501 … 1.0200807590514347 1.1148522647260728; 1.0355761484061679 1.1645743953322452 … 0.9456392877736116 0.97721721648902; … ; 0.9631565516459338 1.0310173173570558 … 0.9397854575550716 1.054789933248645; 1.1726712473288508 0.9513418146829166 … 0.8969359280568239 1.0510540956826318;;; 1.0005057293537893 1.0278840926316877 … 1.0052377945541369 1.031830793368726; 1.039316102506508 1.03384947223874 … 1.0179233256707965 1.0172062002490452; … ; 0.9987645476718533 1.0210445590511399 … 0.9689083005144766 0.9893687386017225; 1.0263575268357814 0.9954653675521066 … 0.9655948885433956 0.9882072293229633], [[1.0;;], [0.13285 0.10619 … 0.1355

In [27]:
res.states

15×9×3 Array{Float64, 3}:
[:, :, 1] =
 1.00849   1.29437   1.44405   0.991208  …  1.03392   1.20453   1.07523
 1.09639   0.966835  0.920865  0.959897     1.23745   1.08559   1.37052
 1.10591   1.18697   0.903144  1.13412      0.980715  0.953924  1.20573
 0.906442  0.977135  1.25606   1.02208      0.896212  1.1215    1.35275
 0.986536  0.902867  1.08172   1.02292      0.891283  1.09784   1.26642
 1.09279   1.2674    1.34166   0.899919  …  1.11331   1.06765   0.980165
 1.24957   1.08824   0.888952  1.11515      0.903853  1.09791   1.01496
 0.965796  0.893123  1.09261   1.0053       0.907803  1.35692   1.23867
 1.08657   1.26008   0.891074  1.33985      1.02282   0.978013  1.11598
 1.00462   1.09881   0.893179  0.972233     1.35817   1.09817   1.24506
 1.35972   0.959547  1.19265   0.954046  …  1.20986   0.966507  1.06042
 1.10558   0.887245  1.24696   1.00925      1.10599   1.36498   0.978803
 0.974185  1.36131   1.10663   0.908443     0.899902  1.09719   1.00793
 1.36672   0.96606   0.8

In [28]:
res.probs

16-element Vector{Matrix{Float64}}:
 [1.0;;]
 [0.13285 0.10619 … 0.13554 0.08543]
 [0.14580353782461422 0.09567181031238238 … 0.15566428302596913 0.10244636808430561; 0.06017515773613335 0.04699124211319333 … 0.15886618325642715 0.08230530181749694; … ; 0.054079976390733364 0.04065220599085141 … 0.15663272834587574 0.07827947469381732; 0.034999414725506266 0.016036521128409222 … 0.1284092239260213 0.1451480744469156]
 [0.15784114052953158 0.057230142566191446 … 0.14134419551934826 0.16364562118126272; 0.14023261961406291 0.16679883690192968 … 0.08220988633359767 0.08683584456780333; … ; 0.12934465375431148 0.04271690103475723 … 0.16954099230565137 0.1601883788803396; 0.14317507418397626 0.07460788469690546 … 0.1599194573972022 0.08011869436201781]
 [0.09262295081967213 0.1823397913561848 … 0.11132637853949329 0.05648286140089419; 0.1939214081047892 0.12453950061399918 … 0.12464183381088825 0.05106426524764634; … ; 0.07345330964702247 0.16150459565922606 … 0.12682474704564764 0.08859195

In [34]:
round.(res.states[5,:,:]; digits=4)

9×3 Matrix{Float64}:
 0.9865  1.0237  1.0264
 0.9029  0.9853  0.9912
 1.0817  0.9102  0.9747
 1.0229  1.1695  1.0366
 1.1153  1.0568  0.9977
 1.3515  0.9371  0.9965
 0.8913  1.1162  1.0076
 1.0978  0.9328  1.0078
 1.2664  1.0463  1.0273

In [35]:
round.(res.states[6,:,:]; digits=4)

9×3 Matrix{Float64}:
 1.0928  0.9327  1.0056
 1.2674  1.0445  1.0264
 1.3417  0.9358  0.9937
 0.8999  0.9906  0.9892
 1.0247  1.1672  1.037
 0.8923  1.1231  1.0068
 1.1133  1.0591  0.9969
 1.0676  0.9112  0.9721
 0.9802  1.0214  1.0247

In [15]:
round.(res.probs[6]; digits=2)

9×9 Matrix{Float64}:
 0.11  0.14  0.04  0.11  0.05  0.18  0.12  0.11  0.13
 0.07  0.06  0.17  0.15  0.14  0.07  0.17  0.13  0.04
 0.13  0.06  0.1   0.1   0.14  0.1   0.17  0.12  0.07
 0.06  0.1   0.15  0.19  0.08  0.09  0.17  0.11  0.05
 0.1   0.09  0.11  0.13  0.11  0.1   0.19  0.12  0.06
 0.07  0.12  0.11  0.18  0.07  0.13  0.13  0.13  0.06
 0.16  0.07  0.06  0.08  0.11  0.14  0.13  0.14  0.1
 0.13  0.1   0.06  0.1   0.07  0.17  0.12  0.15  0.11
 0.16  0.11  0.03  0.08  0.06  0.17  0.12  0.12  0.15

In [36]:
save_object("scenario_grid.jld2", res)

In [37]:
grid = load_object("scenario_grid.jld2")

Lattice([1.0084913942473595 1.2943722619880673 … 1.2045282066834457 1.0752274448581267; 1.0963921326249169 0.9668353671859581 … 1.0855946827898875 1.3705173432588122; … ; 1.3667152684087012 0.9660598526959102 … 0.9503650809004188 1.120501476035486; 0.9840055595486175 1.3637141036185536 … 1.1632620944457561 1.1207337251940046;;; 0.9336306910725175 1.026510299706501 … 1.0200807590514347 1.1148522647260728; 1.0355761484061679 1.1645743953322452 … 0.9456392877736116 0.97721721648902; … ; 0.9631565516459338 1.0310173173570558 … 0.9397854575550716 1.054789933248645; 1.1726712473288508 0.9513418146829166 … 0.8969359280568239 1.0510540956826318;;; 1.0005057293537893 1.0278840926316877 … 1.0052377945541369 1.031830793368726; 1.039316102506508 1.03384947223874 … 1.0179233256707965 1.0172062002490452; … ; 0.9987645476718533 1.0210445590511399 … 0.9689083005144766 0.9893687386017225; 1.0263575268357814 0.9954653675521066 … 0.9655948885433956 0.9882072293229633], [[1.0;;], [0.13285 0.10619 … 0.1355

In [38]:
using  Test
@test grid.states[5,:,:] ≈ res.states[5,:,:]

Test Passed

In [41]:
res.states[3,:,3]

9-element Vector{Float64}:
 1.0143305473204354
 0.9854785572603496
 1.003604484079091
 1.0038924296581662
 1.0107697123646282
 0.9910509301591774
 1.0332650815623272
 1.0259226703370474
 1.034748568861909

#### TODO

- evaluate moments